In [95]:
import cv2
import time
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

GAUSSIAN_BLUR_KERNEL_SIZE = (5, 5)
GAUSSIAN_BLUR_SIGMA_X = 0
CANNY_THRESHOLD1 = 200
CANNY_THRESHOLD2 = 450

browser = webdriver.Chrome()
browser.get('https://www.ghxi.com/login')
browser.find_element(By.CSS_SELECTOR, '.captcha-button').click()
time.sleep(2)                                   # 待改进
browser.switch_to.frame('tcaptcha_iframe')      # 切换到验证码框架

In [59]:
# 根据验证码图片链接获取图片
def by_src():
    image_src           = browser.find_element(By.CSS_SELECTOR, '#slideBg').get_attribute('src')
    image_by_src        = requests.get(image_src).content
    image_by_src        = Image.open(BytesIO(image_by_src))
    return image_by_src


# screenshot_as_png 方法
def by_screenshot():
    slideBgWrap         = browser.find_element(By.CSS_SELECTOR, '#slideBgWrap')
    image_by_screenshot = Image.open(BytesIO(slideBgWrap.screenshot_as_png))
    image_by_screenshot
    return image_by_screenshot

In [85]:
# 高斯滤波
def get_gaussian_blur_image(image):
    return cv2.GaussianBlur(image, GAUSSIAN_BLUR_KERNEL_SIZE, GAUSSIAN_BLUR_SIGMA_X)

# 边缘检测
def get_canny_image(image):
    return cv2.Canny(image, CANNY_THRESHOLD1, CANNY_THRESHOLD2)

# 轮廓提取
def get_contours(image):
    contours, _ = cv2.findContours(image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    return contours


##### 各个范围
# 宽度范围
def w_threshold(image_width):
    min_w = image_width * 0.132 * 0.9
    max_w = image_width * 0.132 * 1.1
    return min_w, max_w

def h_threshold(image_height):
    min_h = image_height * 0.244 * 0.9
    max_h = image_height * 0.244 * 1.1
    return min_h, max_h

# 缺口面积范围（20%误差）
def get_contour_area_threshold(image_width, image_height):
    contour_area_min = (image_width * 0.132) * (image_height * 0.244) * 0.8
    contour_area_max = (image_width * 0.132) * (image_height * 0.244) * 1.2
    print('【面积范围】', contour_area_min, '——', contour_area_max)
    return contour_area_min, contour_area_max

# 缺口周长范围（20%误差）
def get_arc_length_threshold(image_width, image_height):
    arc_length_min = ((image_width * 0.132) + (image_height * 0.244)) * 2 * 0.9
    arc_length_max = ((image_width * 0.132) + (image_height * 0.244)) * 2 * 1.1
    print('【周长范围】', arc_length_min, '——', arc_length_max)
    return arc_length_min, arc_length_max

# 最小偏移与最大偏移？？？
def get_offset_threshold(image_width):
    
    offset_min = 0.24  * image_width     # 缺口位置 最小偏移是验证码宽度的 0.24 倍
    offset_max = 0.868 * image_width     # 缺口位置 最大偏移是验证码宽度的 0.85 倍
    print('【偏移范围】', offset_min, '——', offset_max)
    return offset_min, offset_max

def test(img):
    image_raw                    = np.array(img)
    image_height, image_width, _ = image_raw.shape              # 验证码图片的尺寸（高度、宽度）
    print('【图片】', image_raw.shape)
    ### 目标轮廓的面积、周长、偏移
    contour_area_min, contour_area_max = get_contour_area_threshold(image_width, image_height)
    arc_length_min,   arc_length_max   = get_arc_length_threshold(image_width, image_height)
    min_w, max_w                       = w_threshold(image_width)
    min_h, max_h                       = h_threshold(image_height)
    offset_min,       offset_max       = get_offset_threshold(image_width)
    exception_offset = None
    # 图片处理
    image_gaussian_blur = get_gaussian_blur_image(image_raw)    # 高斯滤波
    image_canny         = get_canny_image(image_gaussian_blur)  # 边缘检测
    contours            = get_contours(image_canny)             # 轮廓提取

    print(type(image_raw))
    print(type(image_canny))
    print(type(Image.fromarray(image_canny.astype('uint8'))))
    
    ### 筛选出符合条件的轮廓 即为 验证码缺口轮廓
    mid_contour = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)  # 各轮廓外框信息
        
        # 绘制外框
        color       = (255, 0, 0)   # 外框颜色
        start_point = (x, y)        # 外框起点
        end_point   = (x+w, y+h)    # 外框终点
        image_area  = w * h         # 外框面积
        image_arc   = (w + h) * 2   # 外框周长
             
        # 条件判断
        if contour_area_min < image_area < contour_area_max and \
            arc_length_min  < image_arc  < arc_length_max and \
            min_w           < w          < max_w and \
            min_h           < h          < max_h and \
            offset_min      < x          < offset_max:
            current_offset  = x
            rec_image         = cv2.rectangle(image_canny, start_point, end_point, color, 4)
            rec_image_finally = Image.fromarray(rec_image.astype('uint8'))
            print('【识别成功】')                                                          # 缺口偏移
            print('【轮廓信息】', (x, y, w, h), '【面积】', image_area, '【周长】', image_arc)
            print('【当前偏移offset】', current_offset)
            # mid_contour.append((x, y, w, h))
            exception_offset = current_offset * 0.5015 - 37
    # print(mid_contour)
   
    print('【期望偏移】', exception_offset)
    return rec_image_finally, exception_offset

# 测试
# test(by_src())[0]

# 滑动

> 官方文档说明
> 
> [**`drag_and_drop_by_offset(source, xoffset, yoffset)`**](https://selenium-python-zh.readthedocs.io/en/latest/api.html#selenium.webdriver.common.action_chains.ActionChains.drag_and_drop_by_offset)

In [98]:
# 如果过期执行此行代码
# 
browser.refresh()                                                   # 刷新页面
browser.find_element(By.CSS_SELECTOR, '.captcha-button').click()    # 点击 “点击进行人机验证” 按钮
time.sleep(2)
browser.switch_to.frame('tcaptcha_iframe')                          # 切换到验证码所在框架

In [101]:
origin_position = browser.find_element(By.CSS_SELECTOR, '#tcaptcha_drag_thumb')
actions = ActionChains(browser)
finally_image, offset = test(by_src())
actions.drag_and_drop_by_offset(origin_position, offset, 0)
# actions.drag_and_drop_by_offset(origin_position, 0, 0)
actions.drag_and_drop_by_offset(origin_position, 50, 0)     # 测试
actions.perform()       # 执行所存储的操作

finally_image           # 图片展示

In [ ]:
地FDA水电费as